In [ ]:
# if code doesnt run make sure the click version is updated
# ! pip install --upgrade click
! pip install spacy
! python -m spacy download en_core_web_sm
! python -m spacy download en

In [ ]:
from usefulclasses import Node, LinkedList, Queue, heapsort
import json
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download("punkt")
nltk.download('stopwords')

import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
stopwords_set = set(stopwords.words('english'))
ps = PorterStemmer()

with open('books.json') as file:
    books = json.load(file)
file.close()
    
with open('users.json') as file:
    users = json.load(file)
file.close()

In [ ]:
class Book:
    stopwords_set = set(stopwords.words('english'))

    def __init__(self, author: str, desc: str, genre: list, isbn: str, pages: int, rating: int, title: str, totalratings: int):
        self.author = author                  
        self.desc = desc                       
        self.genre = genre
        self.isbn = isbn
        self.pages = pages
        self.rating = rating
        self.title = title                      
        self.totalratings = totalratings
        self.rate = self.rating * self.totalratings

    def __str__(self):
        return self.title + " by " + self.author + "\trate: " + str(self.rate)
    
def create_books_list():
    books_list = []
    for book in books:
        author, desc, genre, isbn, pages, rating, title, totalratings = list(book.values())
        obj = Book(author, desc, list({g.lower() for g in genre.split(',')}), isbn, pages, rating, title, totalratings)
        books_list.append(obj)

    heapsort(books_list)
    return books_list

def create_books_dict(books_list):
    verify_word = lambda word: word.lower() not in stopwords_set and word.isalnum()

    tokenized_title = lambda book: {word.lower() for word in book.title.split(' ') if verify_word(word)}
    tokenized_desc = lambda book: {word.lower() for word in book.desc.split(' ') if verify_word(word)}

    return { book.isbn : tokenized_title(book) | tokenized_desc(book) | set(book.genre) for book in books_list }

In [ ]:
users_list = create_books_list()
books_dict = create_books_dict()

size = len(users_list)

In [ ]:
class User:
    
    def __init__(self, id: int, name: str, fav_genres: LinkedList, library: Queue, wishlist: Queue):
        self.id = id
        self.name = name
        self.fav_genres = fav_genres
        self.library = library
        self.wishlist = wishlist
        self.friends = LinkedList()

    def read_book(self):
        pass

    def rate_book(self):
        pass

    def add_to_wishlist(self):
        pass

    def search_author(self, author: str) -> LinkedList:
        books = LinkedList()
        for book in users_list:
            if author in book.author:
                books.prepend(book)
        return books

    def search_book(self, title: str) -> LinkedList:
        books = LinkedList()
        for book in users_list:
            if title in book.title:
                books.prepend(book)
        return books

    def search_book_isbn(self, isbn: str) -> Book:
        for book in users_list:
            if isbn in book.isbn:
                return book

    def search_genre(self, genre: str) -> LinkedList:
        books = LinkedList()
        for book in users_list:
            if genre.lower() in book.genre:
                # prepend to get in decreasing order with low cost of operation (add at head)
                books.prepend(book)
        return books
    
    def add_friends(self, users: list):
        for user in users:
            if user.most_checked_genre() in self.fav_genres:
                self.friends.prepend(user)

    def search_keywords(self, keywords: str, lower = 0, higher = size) -> LinkedList:
        keyterms = [ps.stem(term) for term in keywords.split(',')]
        books = LinkedList()
        for i in range(lower, higher):
            book = users_list[i]
            tokens = [ps.stem(word) for word in books_dict[book.isbn]]
            score = sum([1 if lemma in keyterms else 0 for lemma in tokens])
            # book must have at least 5 keyterms in common
            if score >= 6:
                books.prepend(book.title)
        
        return books

    def _find_max_count(self, data: list) -> str:
        # using count dictionary approach
        counts = dict()
        for elem in data:
            # set count to 0 if checked for the first time else add 1
            counts[elem] = counts.get(elem, 0) + 1

        max_count = 0
        for (elem, count) in counts.items():
            (max_name, max_count) = (elem, count) if max_count < count else (max_name, max_count)

        return max_name

    def most_checked_genre(self) -> str:
        genres = [genre for book in self.library for genre in book.genre]
        if genres != []:
            return self._find_max_count(genres)

    # def most_checked_author(self) -> str:
    #     # same approach as genre
    #     authors = [author for book in self.library for author in book.author.split(',')]
    #     if authors != []:
    #         return self._find_max_count(authors)

    def recommend_books_by_genre(self) -> dict: # popularity based
        books = dict() 
        for genre in self.fav_genres:
            count = 3
            books[genre.val] = LinkedList()
            # traverse books_list (heap) in reverse order to get values in decreasing order
            for i in range(len(users_list) - 1, -1, -1):
                if genre.val in users_list[i].genre and count != 0:
                    books[genre.val].append(users_list[i])
                    count -= 1
        return books
    
    def recommend_books(self): # content based
        recommended = LinkedList()
        
        liked_books = [book for (book, rate) in self.library if rate >= 4]
        for book in liked_books:
            idx = users_list.idx(book)
            higher, lower = min(idx + 20, size), max(idx - 20, 0)
            recs = self.search_keywords(books_dict[book.isbn], lower, higher)
            recommended.prepend(recs)
            
        return recommended
    
    def recommend_other_users(self):
        experiment = LinkedList()
        for friend in self.friends:
            friend_books = [book for (book, rate) in friend.library if book not in self.library and rate >= 4]
            experiment.prepend(friend_books)
        


In [ ]:
def create_users_list():
    users_list = []
    for user in users:
        user_id, name, genres = list(user.values())
        obj = User(user_id, name, genres)
        users_list.append(obj)

    return users_list

users = create_users_list()